In [1]:
"""
利用VLLM部署Qwen大模型，并获取结构化输出

"""

import json
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional, List, Annotated




In [2]:
# 部署qwen的命令
# !python -m vllm.entrypoints.openai.api_server --served-model-name Qwen/Qwen2-7B-Instruct-AWQ  --model Qwen/Qwen2-7B-Instruct-AWQ --quantization awq --gpu-memory-utilization 0.4 --max-model-len 2048 


In [3]:

# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

class NameLocation(BaseModel):

    names: Annotated[List[str], Field(description="抽取所有的姓名")]
    citys: Annotated[List[str], Field(description="抽取所有的城市")]

chat_response = client.chat.completions.create(
    model="Qwen/Qwen2-7B-Instruct-AWQ",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "抽取下面query中的姓名和城市。\nquery：张三和李四都喜欢上海"},
    ],
    temperature=0.01,
    extra_body={"guided_json":NameLocation.model_json_schema()}  # https://github.com/vllm-project/vllm/blob/v0.6.0/vllm/engine/arg_utils.py#L276
)


result = json.loads(chat_response.choices[0].message.content)
print(result)



{'names': ['张三', '李四'], 'citys': ['上海']}
